In [1]:
push!(LOAD_PATH, ".")
using Power
using Traffic
using Hydrogen
using JLD

In [2]:
##########################################
# info
##########################################
# HRS connect to grid bus number
HRS_bus = [2, 5, 8, 9, 12, 16, 18, 21] 
# HRS location is stored in "Traffic.jl"

# power factor of electrolyser
power_factor = 0.95
QP_ratio = sqrt(1-power_factor^2) / power_factor;

In [3]:
# initial value for iteration
T = 24
# 1. TA and LMP 
# the initial power of electrolyser is 0
# the initial price of hydrogen is average price ($10)
ρⱼ = 10 * ones((Nₕ, T))
pᵈʰ = 0 * ones((length(bus), T))
pᵈʰ[HRS_bus, :] = zeros((Nₕ, T))
qᵈʰ = QP_ratio * pᵈʰ;

record_ρⱼ = []
record_pᵈ = []
record_λⱼ = []
record_hydrogen_demand = []
record_true_ρⱼ = []

# save path 
output_path = "./result/record.jld"

# continue from the last run
if isfile(output_path)
    last_run = load(output_path)
    # get the last results
    record_ρⱼ = last_run["record_ρⱼ"]
    record_pᵈ = last_run["record_pᵈ"]
    record_λⱼ = last_run["record_λⱼ"]
    record_true_ρⱼ = last_run["record_true_ρⱼ"]
    record_hydrogen_demand = last_run["record_hydrogen_demand"]
    ρⱼ = last_run["record_ρⱼ"][end]
    pᵈ = last_run["record_pᵈ"][end]
    λⱼ = last_run["record_λⱼ"][end]
    hydrogen_demand = last_run["record_hydrogen_demand"][end]
    # rearrange pᵈʰ, qᵈʰ, ρⱼ
    pᵈʰ = 0 * ones((length(bus), T))
    pᵈʰ[HRS_bus, :] = pᵈ  # assign power to bus
    qᵈʰ = QP_ratio * pᵈʰ  # assign power to bus
end

In [4]:
N = 1000
for n in 1:N
    # true iteration times
    iter_n = length(record_ρⱼ) + 1
    
    # T single-period TA and LMP problem
    λⱼ = zeros((Nₕ, T))
    hydrogen_demand = zeros((Nₕ, T))
    for t = 1:T
        println("------------------------------------------------")
        println("n=$n, iter_n=$iter_n, t=$t start")
        λⱼ[:, t] = solve_LMP(pᵈᵗAllTime[:,t], qᵈᵗAllTime[:,t], pᵈʰ[:,t], qᵈʰ[:,t])[HRS_bus]
        hydrogen_demand[:, t] = solve_TA(ρⱼ, qᵍALLTime[t], qʰALLTime[t])
        println("n=$n, iter_n=$iter_n, t=$t finish")
        println("------------------------------------------------")
    end

    # multi-period HRS operation problem
    HRS_result = solve_HRS_op(λⱼ, hydrogen_demand)

    # rearrange pᵈʰ, qᵈʰ, ρⱼ
    pᵈʰ = 0 * ones((length(bus), T))
    pᵈʰ[HRS_bus, :] = HRS_result.pᵈ  # assign power to bus
    qᵈʰ = QP_ratio * pᵈʰ  # assign power to bus
    # inertial hydrogen price
    if length(record_ρⱼ) > 1
        factor = 0.1
        ρⱼ = record_ρⱼ[end]  + factor * (HRS_result.ρⱼ - record_ρⱼ[end])
    else
        ρⱼ = HRS_result.ρⱼ
    end
    
    # record all variabels
    push!(record_pᵈ, HRS_result.pᵈ)
    push!(record_true_ρⱼ, HRS_result.ρⱼ)
    push!(record_ρⱼ, ρⱼ)
    push!(record_λⱼ, λⱼ)
    push!(record_hydrogen_demand, hydrogen_demand)

    # save variables
    save(output_path, "record_ρⱼ", record_ρⱼ, "record_pᵈ", record_pᵈ, 
        "record_λⱼ", record_λⱼ, "record_hydrogen_demand", record_hydrogen_demand,
        "record_true_ρⱼ", record_true_ρⱼ
    )

    # exit condition
    if length(record_hydrogen_demand) > 1
        # absolute errors
        error_ρⱼ = sum(abs.(record_ρⱼ[end] - record_ρⱼ[end-1]))
        error_pᵈ = sum(abs.(record_pᵈ[end] - record_pᵈ[end-1]))
        error_λⱼ = sum(abs.(record_λⱼ[end] - record_λⱼ[end-1]))
        error_hydrogen_demand = sum(abs.(record_hydrogen_demand[end] - record_hydrogen_demand[end-1]))
        error = error_ρⱼ + error_pᵈ + error_λⱼ + error_hydrogen_demand
        println("*********")
        println("n=$n, iter_n=$iter_n")
        println("Error: $error")
        println("- error_ρⱼ: $error_ρⱼ") 
        println("- error_pᵈ: $error_pᵈ") 
        println("- error_λⱼ: $error_λⱼ")
        println("- error_hydrogen_demand: $error_hydrogen_demand")
        println("*********")
        
        # relative errors
        error_ρⱼ = sum(abs.(record_ρⱼ[end] - record_ρⱼ[end-1])) / sum(abs.(record_ρⱼ[end]))
        error_pᵈ = sum(abs.(record_pᵈ[end] - record_pᵈ[end-1]))  / sum(abs.(record_pᵈ[end]))
        error_λⱼ = sum(abs.(record_λⱼ[end] - record_λⱼ[end-1]))  / sum(abs.(record_λⱼ[end]))
        error_hydrogen_demand = sum(abs.(record_hydrogen_demand[end] - record_hydrogen_demand[end-1]))  / sum(abs.(record_hydrogen_demand[end]))
        error = error_ρⱼ + error_pᵈ + error_λⱼ + error_hydrogen_demand
        println("*********")
        println("n=$n, iter_n=$iter_n")
        println("Error: $error")
        println("- relative error_ρⱼ: $error_ρⱼ") 
        println("- relative error_pᵈ: $error_pᵈ") 
        println("- relative error_λⱼ: $error_λⱼ")
        println("- relative error_hydrogen_demand: $error_hydrogen_demand")
        println("*********")
        
        # exit condition for relative errors
        if error_ρⱼ < 1e-5 && error_pᵈ < 1e-2 && error_λⱼ < 1e-5 && error_hydrogen_demand < 1e-5
            println("Converged!")
            break
        end
    end
end


------------------------------------------------
n=1, iter_n=1, t=1 start
Academic license - for non-commercial use only
【Power Result】:Model was solved to optimality (subject to tolerances), and an optimal solution is available.

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

【Traffic Result】:Solve_Succeeded
n=1, iter_n=1, t=1 finish
------------------------------------------------
------------------------------------------------
n=1, iter_n=1, t=2 start
Academic license - for non-commercial use only
【Power Result】:Model was solved to optimality (subject to tolerances), and an optimal solution is available.
【Traffic Result】:Solve_Succeeded
n=1, iter_n=

【Power Result】:Model was solved to optimality (subject to tolerances), and an optimal solution is available.
【Traffic Result】:Solve_Succeeded
n=1, iter_n=1, t=24 finish
------------------------------------------------


8×24 Matrix{Float64}:
  9.96408  9.95934   9.96408   9.972    …   9.97329   9.97902   9.97204
  9.9337   9.93659   9.93386   9.93232      9.93143   9.93067   9.93217
  9.93068  9.93633   9.93084   9.92247      9.91399   9.91541   9.9225
  9.99462  9.98222   9.99429  10.0108      10.052    10.0548   10.011
 10.0112   9.99908  10.0112   10.045       10.0622   10.046    10.045
  9.94209  9.93659   9.94225   9.93924  …   9.92714   9.92816   9.93908
 10.0051   9.99516  10.0051   10.0136      10.01     10.0034   10.0177
  9.98558  9.98561   9.98541   9.96674      9.96366   9.96432   9.96287

8×24 Matrix{Float64}:
 4.32686   3.44056e-9  5.29575     …  7.4281        1.87578     5.13179
 0.269406  4.00255     1.79888e-8     5.27318       3.62847e-8  3.35028
 0.134791  1.29087     1.49207        3.1422e-9     1.11244     2.91849
 5.69158   0.641067    7.02674        5.54541       7.96077     6.87409
 3.73026   6.7946      2.3803         6.00151      10.2692      5.69198
 0.644275  1.30255     4.70155     …  5.47654e-10   2.26964     3.65916
 3.46106   6.6193      2.11101        6.3693        8.36646     4.47394
 5.2876    3.49294     3.93032        1.59803       6.61909     2.02224

Academic license - for non-commercial use only
【Hydrogen Result】:Model was solved to optimality (subject to tolerances), and an optimal solution is available.
Solution is optimal
------------------------------------------------
n=2, iter_n=2, t=1 start
Academic license - for non-commercial use only
【Power Result】:Model was solved to optimality (subject to tolerances), and an optimal solution is available.
【Traffic Result】:Solve_Succeeded
n=2, iter_n=2, t=1 finish
------------------------------------------------
------------------------------------------------
n=2, iter_n=2, t=2 start
Academic license - for non-commercial use only
【Power Result】:Model was solved to optimality (subject to tolerances), and an optimal solution is available.
【Traffic Result】:Solve_Succeeded
n=2, iter_n=2, t=2 finish
------------------------------------------------
------------------------------------------------
n=2, iter_n=2, t=3 start
Academic license - for non-commercial use only
【Power Result】:Model was

...

【Traffic Result】:Solve_Succeeded
n=9, iter_n=9, t=24 finish
------------------------------------------------
Academic license - for non-commercial use only
【Hydrogen Result】:Model was solved to optimality (subject to tolerances), and an optimal solution is available.
Solution is optimal
*********
n=9, iter_n=9
Error: 26.60502097425254
- error_ρⱼ: 0.02679766084763635
- error_pᵈ: 5.662011059972367
- error_λⱼ: 0.16740559121379306
- error_hydrogen_demand: 20.748806662218744
*********
*********
n=9, iter_n=9
Error: 0.00830778087673188
- relative error_ρⱼ: 1.397740867980444e-5
- relative error_pᵈ: 0.0067942179217348
- relative error_λⱼ: 5.714419053053207e-6
- relative error_hydrogen_demand: 0.0014938711272642227
*********
------------------------------------------------
n=10, iter_n=10, t=1 start
Academic license - for non-commercial use only
【Power Result】:Model was solved to optimality (subject to tolerances), and an optimal solution is available.
【Traffic Result】:Solve_Succeeded
n=10, ite

【Power Result】:Model was solved to optimality (subject to tolerances), and an optimal solution is available.
【Traffic Result】:Solve_Succeeded
n=10, iter_n=10, t=23 finish
------------------------------------------------
------------------------------------------------
n=10, iter_n=10, t=24 start
Academic license - for non-commercial use only
【Power Result】:Model was solved to optimality (subject to tolerances), and an optimal solution is available.

8×24 Matrix{Float64}:
  9.98016   9.97592   9.98008   9.99542  …   9.99641  10.0113    9.99442
  9.95184   9.95556   9.95193   9.95078      9.96721   9.96528   9.95071
  9.94937   9.95606   9.95162   9.94781      9.93707   9.94078   9.94886
  9.98975   9.98412   9.98937  10.0045      10.0209   10.0185   10.005
 10.015    10.0071   10.0149   10.0428      10.0626   10.0426   10.0246
  9.95401   9.95779   9.95409   9.94911  …   9.92977   9.93247   9.94904
 10.0143   10.0072   10.0121   10.0019      10.016    10.0079   10.002
  9.99516   9.99219   9.99494   9.9743       9.9731    9.97338   9.99294

8×24 Matrix{Float64}:
 4.5816    2.8359e-9  5.69652      …  5.3947       5.68318   5.79404
 0.549658  4.20699    2.26195e-10     6.7211       0.818163  3.73758
 0.433204  1.53035    1.89238         3.91149e-9   2.26868   3.65044
 5.03286   0.150425   6.3683          3.84675      6.02397   6.29072
 3.51923   6.63102    2.16923         5.80745      9.8566    4.51438
 0.651553  1.61213    4.70883      …  6.40281e-10  1.53437   3.65916
 3.49012   6.63686    2.03822         6.31412      8.22281   3.4513
 5.2876    3.23009    3.93032         1.59803      6.59904   3.02433


【Traffic Result】:Solve_Succeeded
n=10, iter_n=10, t=24 finish
------------------------------------------------
Academic license - for non-commercial use only
【Hydrogen Result】:Model was solved to optimality (subject to tolerances), and an optimal solution is available.
Solution is optimal
*********
n=10, iter_n=10
Error: 4.518336862230493
- error_ρⱼ: 0.01695595256646598
- error_pᵈ: 4.4170884773146835
- error_λⱼ: 0.08419340248087792
- error_hydrogen_demand: 9.902986846554995e-5
*********
*********
n=10, iter_n=10
Error: 0.005312078947054854
- relative error_ρⱼ: 8.844080884424745e-6
- relative error_pᵈ: 0.005300353776251109
- relative error_λⱼ: 2.8739599737918885e-6
- relative error_hydrogen_demand: 7.1299455283105096e-9
*********
Converged!
